# SETUP

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [2]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/sports/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Sports</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Sports" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Sports&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" /><meta pro

In [3]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Sports  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/sports/"+queryString+myHash));  (function(){("und

## EXTRACT TITLES

In [4]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [5]:
summaries[0]

<div class="flowItem feature" entity="CMS-ExternalLink_3853429"><a href="https://sport.francetvinfo.fr/football/football-platini-deboute-par-la-cedh-qui-juge-sa-suspension-justifiee" rel="noopener" target="_blank"><figure id="image_3649739"><img alt="Michel Platini lors des funérailles de l\'ancien président de l\'UEFA, Lennart Johansson, le 26 juin 2019 à Stockholm (Suède)." botag="CMS-IMAGE_3649739" class="b-lazy" data-src="/image/75rw41zng-c0ae/840/472/21077621.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="Michel Platini lors des funérailles de l'ancien président de l'UEFA, Lennart Johansson, le 26 juin 2019 à Stockholm (Suède). | PATRIK OSTERBERG / TT NYHETSBYR?N" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">Football : la CEDH déboute Michel Platini et juge que sa suspension n'était "ni excessive ni arbitraire"</h2><div class="description" field="description">L'ancien patron de l'UEF

In [6]:
summaries[0].select_one("h2").get_text()

'Football : la CEDH déboute Michel Platini et juge que sa suspension n\'était "ni excessive ni arbitraire"'

In [7]:
len(summaries)

30

In [8]:
sport_news = [x.select_one("h2").get_text() for x in summaries]

In [9]:
sport_news

['Football : la CEDH déboute Michel Platini et juge que sa suspension n\'était "ni excessive ni arbitraire"',
 "La médaille du jour. En Italie, à cause du coronavirus, un fan de foot fait 13\xa0000km pour voir un match... qui n'a pas eu lieu !",
 'VIDEOS. Coupe de France : le PSG se qualifie pour la finale en battant Lyon (5-1)',
 "DIRECT. Coupe de France : le PSG enfonce le clou face à l'OL réduit à 10 (5-1)",
 'Quatre candidatures officiellement validées pour le poste de président de la Fédération française des sports de glace',
 'Coronavirus : la quarantaine des équipes cyclistes Cofidis, Groupama-FDJ et Gazprom-RusVelo prolongées "jusqu\'au 14 mars" à Abu Dhabi',
 'La médaille du jour. Hommage à Nicolas Portal',
 'Violences sexuelles dans le patinage : "On a toujours su qu\'il y avait des mentalités assez misogynes, mais pas à ce point-là", affirme Gabriella Papadakis',
 "Le directeur sportif d'Ineos Nicolas Portal meurt à 40 ans d'une crise cardiaque",
 'Ligue des Nations : la Fra

# WRITE THE DATA IN POSTGRESQL DB

In [10]:
df = pd.DataFrame({"source": "franceinfo", "headline": sport_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [11]:
df.head()

,source,headline,date
0,franceinfo,Football : la CEDH déboute Michel Platini et j...,2020-03-05
1,franceinfo,"La médaille du jour. En Italie, à cause du cor...",2020-03-05
2,franceinfo,VIDEOS. Coupe de France : le PSG se qualifie p...,2020-03-05
3,franceinfo,DIRECT. Coupe de France : le PSG enfonce le cl...,2020-03-05
4,franceinfo,Quatre candidatures officiellement validées po...,2020-03-05


In [12]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [13]:
conn = alchemy_engine.connect()
table = "sport"

In [14]:
try:
    frame = df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:d
    print(ex)
conn.close()